# Imports
---
Import new skis / snowboards to an SQL database

In [520]:
import json
import pymysql
import re

In [521]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../app/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

In [522]:
db = setupdb()
cursor = db.cursor()

In [523]:
params = {
    'size': ['size', 'length'],
    'nose width': ['nose width', 'tip width'],
    'waist width': ['waist width'],
    'tail width': ['tail width'],
    'sidecut': ['sidecut radius', 'turning radius', 'radius', 'sidecut'],
    'setback': ['stance setback', 'setback'],
    'effective edge': ['effective edge', 'running length']
}

In [524]:
values = {
    'size': '',
    'nose width': '',
    'waist width': '',
    'tail width': '',
    'sidecut': '',
    'setback': '',
    'effective edge': ''
}

In [525]:
# Extraction of data 
# from string format to a tabular structure
# --------------------------------------------------
def extract_data(table):
    # Remove units from headings
    while re.search(r'\(([a-zA-Z]+)\)', table) != None:
        match = re.search(r'\(([a-zA-Z]+)\)', table)
        substr = table[match.start():match.end()]
        table = table.replace(substr, '')
    '''    
    while ')' in table:
        table = table[:table.find('(')] + table[table.find(')') + 1:]
    '''    
        
    breakpoints = []
    
    # Find location of headings in table
    for i in params:
        # Check each param alias
        for j in params[i]:
            if j in table:
                breakpoints.append(table.find(j))
                break
        
        values[i] = []
        
    # Extract table data into rows
    rows = []
    breakpoints.sort()
    for i, b in enumerate(breakpoints):
        if i > 0:
            rows.append(table[breakpoints[i-1]:b])
            print(f"Row: {table[breakpoints[i-1]:b]}")
        if i == len(breakpoints) -1:
            rows.append(table[b:])
    
    
    # Remove lables from rows
    for row in rows:
        for i in params:
            for j in params[i]:
                if j in row:
                    values[i] = row.replace(j, '')
                    break
                    
    
    # Format table rows as lists
    for val in values:
        try:
            # \u0020\u200b\u002f\u0020
            # \u0020\u002f\u0020
            values[val] = values[val].strip()
            values[val] = values[val].replace('\u0020\u002f\u0020', '/')
            values[val] = values[val].replace('\u0020\u200b\u002f\u0020', '/')
            values[val] = values[val].replace('\u0020\u200b\u200b\u002f\u0020', '/')
            values[val] = values[val].split() 
            
        except Exception as e:
            x = e
            
    
    # Truncate long rows to the correct size
    num_sizes = len(values['size'])
    for val in values:
        # Sidecuts can have more than one radius
        if val != "sidecut":
            values[val] = values[val][:num_sizes]

        # Remove rogue characters from row values
        if len(values[val]) > 0 and type(values[val][0]) == str:
            for x, v in enumerate(values[val]):
                values[val][x] = values[val][x].replace('\u200b', '')
        
        
        # Fill empty or missing data with null values
        for x in range(num_sizes - len(values[val])):
            
            values[val].append(0)
            
    
        
        
    return values

In [526]:
# Save skiboard in MySQL DB
# --------------------------------------------------
def save_skiboard(skiboard):
    try:
        sql = f"""INSERT INTO skiboards (
            url,
            brand,
            model,
            year,
            name,
            slug,
            category,
            family,
            description,
            stiffness,
            shape,
            flex_profile,
            camber_profile,
            camber_details,
            core,
            core_profiling,
            fibreglass,
            laminates,
            resin,
            base,
            edges,
            edge_tech,
            topsheet,
            sidewall,
            inserts,
            asym,
            weight,
            womens,
            youth
        ) VALUES (
            '{skiboard['url']}',
            '{skiboard['brand']}',
            '{skiboard['model']}',
            '{skiboard['year']}',
            '{str(skiboard['brand']).title()} {str(skiboard['model']).title()} {str(skiboard['year'])}',
            '{str(skiboard['brand']).lower().replace(' ', '-')}-{str(skiboard['model']).lower().replace(' ', '-')}-{str(skiboard['year'])}',
            '{skiboard['category']}',
            '{skiboard['family']}',
            '{skiboard['description']}',
            '{skiboard['stiffness']}',
            '{skiboard['shape']}',
            '{skiboard['flex_profile']}',
            '{skiboard['camber_profile']}',
            '{skiboard['camber_details']}',
            '{skiboard['core']}',
            '{skiboard['core_profiling']}',
            '{skiboard['fibreglass']}',
            '{skiboard['laminates']}',
            '{skiboard['resin']}',
            '{skiboard['base']}',
            '{skiboard['edges']}',
            '{skiboard['edge_tech']}',
            '{skiboard['topsheet']}',
            '{skiboard['sidewall']}',
            '{skiboard['inserts']}',
            '{skiboard['asym']}',
            '{skiboard['weight']}',
            '{skiboard['womens']}',
            '{skiboard['youth']}'
        )"""
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(f"ERROR:\n{e}")

In [527]:
# Save data in MySQL DB
# --------------------------------------------------
def save_values(skiboard, values):
    #print(f"Importing...\n{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}\nValues:\n{values}")

    for x in range(len(values['size'])):
        try:
            sql = f"""REPLACE INTO sizes (
                skiboard_id,
                size,
                nose_width,
                waist_width,
                tail_width,
                sidecut,
                setback,
                effective_edge
            ) VALUES (
                '{str(skiboard['id'])}',
                '{str(values['size'][x])}',
                '{float(values['nose width'][x])}',
                '{float(values['waist width'][x])}',
                '{float(values['tail width'][x])}',
                '{str(values['sidecut'][x])}',
                '{float(values['setback'][x])}',
                '{float(values['effective edge'][x])}'
            )"""

            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(f"ERROR: size ({values['size'][x]})\n{e}\n")
            print(f"SQL:\n{sql}\n\n")
    
    return True

# Create New SkiBoard
---

In [528]:
id_counter = 180

In [529]:
print(id_counter)

180


In [543]:
skiboard = {
    'id': id_counter,
    'url': '',
    'brand': 'Rome',
    'model': 'Mod',
    'year': 2020,
    'category': 'Snowboard',
    'family': '',
    'description': "Send your boring board to The Hague. The Rome Warden Snowboard rips groomers, polices the tree runs, and liberates you from the cruelty of having to pay for the privilege to rely on someone else's beat-up rental junk. Lay down a mean euro, play in a secret pow stash and slash your way down the hill with authori-tah. The Rome Warden Snowboard hates to be confined and loves to get after it day after day thanks to its durable extruded base and bamboo stringer construction.".replace("'", "`"),
    'stiffness': 8,
    'shape': 'Twin',
    'flex_profile': '',
    'camber_profile': 'Full Camber',
    'camber_details': 'Stay Positive Camber',
    'core': "Pop Core Matrix".replace("'", "`"),
    'core_profiling': '',
    'fibreglass': 'Biax',
    'laminates': 'TurboRod Configuration ~ Glasspack Impact Plates',
    'base': 'SinterCarbon Base',
    'edges': '',
    'edge_tech': '',
    'topsheet': '',
    'resin': '',
    'sidewall': '',
    'inserts': '4x2',
    'asym': 0,
    'weight': 0,
    'womens': 0,
    'youth': 0
}

In [544]:
save_skiboard(skiboard)

In [545]:
print(skiboard['id'])

182


# Add Sizes to SkiBoard
---

In [546]:
#skiboard['id'] = 181

In [547]:
table = input("Please submit yourski/snowboard size table\n\n").lower()

Please submit yourski/snowboard size table

Size (cm)	149	152	155	157W	158	160W Effective Edge (mm)	1150	1180	1210	1230	1240	1260 Waist Width (mm)	248	251	254	268	256	270 Sidecut Radius (m)	7.3	7.52	7.78	7.87	7.93	7.98 Stance Setback (in)	0	0	0	0	0	0 Stance Setback (mm)	0	0	0	0	0	0 Stance Range (in)	18.5 - 23.25	18.5 - 23.25	19.5 - 24.25	19.5 - 24.25	19.5 - 24.25	19.5 - 24.25 Rider Weight (lbs)	100 - 150	110 - 165	115 - 170	130 - 190	150 - 220​+	150 - 220​+ Width	Regular	Regular	Regular	Wide	Regular	Wide


In [548]:
values = extract_data(table)

Row: size 	149	152	155	157w	158	160w 
Row: effective edge 	1150	1180	1210	1230	1240	1260 
Row: waist width 	248	251	254	268	256	270 
Row: sidecut radius 	7.3	7.52	7.78	7.87	7.93	7.98 


In [549]:
for row in values:
    print(f"{row}:\n{values[row]}\n\n")

size:
['149', '152', '155', '157w', '158', '160w']


nose width:
[0, 0, 0, 0, 0, 0]


waist width:
['248', '251', '254', '268', '256', '270']


tail width:
[0, 0, 0, 0, 0, 0]


sidecut:
['7.3', '7.52', '7.78', '7.87', '7.93', '7.98']


setback:
['0', '0', '0', '0', '0', '0']


effective edge:
['1150', '1180', '1210', '1230', '1240', '1260']




In [550]:
success = save_values(skiboard, values)
if success:
    print("\nSUCCESS!")
    id_counter += 1


SUCCESS!
